<a href="https://colab.research.google.com/github/carloslme/data-engineering/blob/main/bucket_migration_s3_to_endpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
This notebook has the code to perform a bucket migration from AWS S3 to an endpoint, for this case, Wasabisys (also called Wasabi).

# Contents
* Prerequisites
* Step 1: Import libraries
* Step 2: Setting credentials
* Step 3: Declare functions 
* Step 4: Get the S3 objects in the buckets
* Step 5: Start migration

# Prerequisites
Before you begin, be sure that you've installed `boto3`

In [1]:
!pip install boto3

     |████████████████████████████████| 131 kB 5.0 MB/s 
     |████████████████████████████████| 79 kB 6.2 MB/s 
     |████████████████████████████████| 7.8 MB 38.1 MB/s 
     |████████████████████████████████| 138 kB 45.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.6 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
